In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML
from imblearn.over_sampling import SMOTE

# Set plotting style
%matplotlib inline
plt.style.use('ggplot')
sns.set_palette('Blues')

# Print working directory and files
print("Current working directory:", os.getcwd())
print("Files in directory:", os.listdir('.'))

Current working directory: c:\Users\khale\OneDrive\Desktop\dataset
Files in directory: ['.git', 'assignments.csv', 'assignment_submissions.csv', 'attendances.csv', 'courses.csv', 'enrollments.csv', 'grades.csv', 'Logistic Regression&Random forest.ipynb', 'merged_data.csv', 'past models', 'users.csv']


In [7]:


# Load data
grades = pd.read_csv('grades.csv')
attendances = pd.read_csv('attendances.csv')
courses = pd.read_csv('courses.csv')

# Simulate fail cases
grades['status'] = grades['total'].apply(lambda x: 'fail' if x < 110 else 'pass')
print("Target distribution in grades.csv after simulation:")
print(grades['status'].value_counts(normalize=True))

# Calculate attendance percentage per student-course
present_count = attendances[attendances['status'].isin(['present', 'late'])].groupby(['student_id', 'course_id']).size().reset_index(name='present_count')
total_sessions = attendances.groupby(['student_id', 'course_id']).size().reset_index(name='total_sessions')
print("Total sessions per student-course:")
print(total_sessions['total_sessions'].describe())

# Merge data
data = grades.merge(present_count, on=['student_id', 'course_id'], how='left')
data = data.merge(total_sessions, on=['student_id', 'course_id'], how='left')
data = data.merge(courses[['id', 'difficulty', 'department']], left_on='course_id', right_on='id', how='left')

# Handle missing values
data['present_count'] = data['present_count'].fillna(0)
data['total_sessions'] = data['total_sessions'].fillna(0)  # Changed from 10 to 0 for accuracy
data['attendance_percentage'] = (data['present_count'] / data['total_sessions']) * 100
data['attendance_percentage'] = data['attendance_percentage'].clip(0, 100)
data['difficulty'] = data['difficulty'].map({'easy': 0, 'medium': 1, 'hard': 2})
data['department'] = data['department'].astype('category').cat.codes
data[['quiz1', 'quiz2', 'midterm', 'assignments']] = data[['quiz1', 'quiz2', 'midterm', 'assignments']].fillna(data[['quiz1', 'quiz2', 'midterm', 'assignments']].median())
data[['difficulty', 'department']] = data[['difficulty', 'department']].fillna({'difficulty': 1, 'department': data['department'].mode()[0]})

# Filter valid status
data = data[data['status'].isin(['pass', 'fail'])]
print("Data shape:", data.shape)
print("Target distribution after merging:")
print(data['status'].value_counts(normalize=True))
print("Attendance percentage summary:")
print(data['attendance_percentage'].describe())

# Select features
feature_names = ['attendance_percentage', 'assignments', 'midterm', 'quiz1', 'quiz2', 'difficulty', 'department']
X = data[feature_names]
y = data['status'].map({'pass': 1, 'fail': 0})

# Check data
print("Missing values in features:\n", X.isna().sum())
print("Feature data types:\n", X.dtypes)
print("Feature ranges:\n", X.describe())



Target distribution in grades.csv after simulation:
status
pass    0.929804
fail    0.070196
Name: proportion, dtype: float64
Total sessions per student-course:
count    34204.000000
mean        30.506374
std          4.767462
min         20.000000
25%         32.000000
50%         32.000000
75%         32.000000
max         52.000000
Name: total_sessions, dtype: float64
Data shape: (34204, 20)
Target distribution after merging:
status
pass    0.929804
fail    0.070196
Name: proportion, dtype: float64
Attendance percentage summary:
count    34204.000000
mean        75.005319
std          7.966090
min         34.375000
25%         68.750000
50%         75.000000
75%         81.250000
max        100.000000
Name: attendance_percentage, dtype: float64
Missing values in features:
 attendance_percentage    0
assignments              0
midterm                  0
quiz1                    0
quiz2                    0
difficulty               0
department               0
dtype: int64
Feature dat

In [8]:


# Assume X and y are defined from Cell 2
if X is not None and y is not None:
    print("Target distribution in data:")
    print(y.value_counts(normalize=True))

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split with stratification
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
    print("Training set target distribution:")
    print(pd.Series(y_train).value_counts(normalize=True))

    # Apply SMOTE
    from imblearn.over_sampling import SMOTE
    smote = SMOTE(sampling_strategy=0.5, random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
    print("Balanced training set distribution:", pd.Series(y_train_balanced).value_counts(normalize=True))

    # Train Logistic Regression
    param_grid_lr = {
        'C': [0.01, 0.1, 1],
        'solver': ['lbfgs'],
        'class_weight': ['balanced']
    }
    grid_lr = GridSearchCV(LogisticRegression(max_iter=5000), param_grid_lr, cv=5, scoring='f1_macro', n_jobs=-1)
    grid_lr.fit(X_train_balanced, y_train_balanced)
    print("Logistic Regression - Best parameters:", grid_lr.best_params_)
    print("Logistic Regression - Best cross-validation F1-score:", grid_lr.best_score_)
    y_prob_lr = grid_lr.predict_proba(X_test)[:, 1]
    y_pred_lr = (y_prob_lr >= 0.5).astype(int)
    print("\nLogistic Regression - Test Accuracy (threshold=0.5):", accuracy_score(y_test, y_pred_lr))
    print("Logistic Regression - Classification Report (threshold=0.5):")
    print(classification_report(y_test, y_pred_lr, target_names=['Fail', 'Pass']))

    # Train Random Forest
    model_rf = RandomForestClassifier(random_state=42)
    param_grid_rf = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'class_weight': ['balanced', None]
    }
    grid_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, scoring='f1_macro', n_jobs=-1)
    grid_rf.fit(X_train_balanced, y_train_balanced)
    model = grid_rf.best_estimator_  # Use Random Forest as the final model
    print("\nRandom Forest - Best parameters:", grid_rf.best_params_)
    print("Random Forest - Best cross-validation F1-score:", grid_rf.best_score_)
    y_prob_rf = model.predict_proba(X_test)[:, 1]
    y_pred_rf = (y_prob_rf >= 0.5).astype(int)
    print("Random Forest - Test Accuracy (threshold=0.5):", accuracy_score(y_test, y_pred_rf))
    print("Random Forest - Classification Report (threshold=0.5):")
    print(classification_report(y_test, y_pred_rf, target_names=['Fail', 'Pass']))
else:
    print("Error: X or y is not defined. Ensure Cell 2 runs successfully first.")

Target distribution in data:
status
1    0.929804
0    0.070196
Name: proportion, dtype: float64
Training set target distribution:
status
1    0.929796
0    0.070204
Name: proportion, dtype: float64
Balanced training set distribution: status
1    0.666667
0    0.333333
Name: proportion, dtype: float64
Logistic Regression - Best parameters: {'C': 0.01, 'class_weight': 'balanced', 'solver': 'lbfgs'}
Logistic Regression - Best cross-validation F1-score: 0.8195591553242652

Logistic Regression - Test Accuracy (threshold=0.5): 0.8098231252740827
Logistic Regression - Classification Report (threshold=0.5):
              precision    recall  f1-score   support

        Fail       0.24      0.82      0.38       480
        Pass       0.98      0.81      0.89      6361

    accuracy                           0.81      6841
   macro avg       0.61      0.81      0.63      6841
weighted avg       0.93      0.81      0.85      6841


Random Forest - Best parameters: {'class_weight': None, 'max_dep

In [9]:


# Assume data is defined from Cell 2
valid_pairs = data[['student_id', 'course_id']].drop_duplicates().sort_values(['student_id', 'course_id'])
print(f"Total unique pairs: {len(valid_pairs)}")

# Create a scrollable HTML table with compact styling
html = valid_pairs.to_html(classes='compact-table', index=False)
html = f"""
<style>
.compact-table {{
    font-size: 10px;
    line-height: 1;
    max-height: 400px;
    overflow-y: auto;
    display: block;
}}
.compact-table th, .compact-table td {{
    padding: 2px;
    border: 1px solid #ddd;
}}
</style>
{html}
"""
display(HTML(html))

Total unique pairs: 34204


student_id,course_id
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10


In [10]:


# Set display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('future.no_silent_downcasting', True)  # Opt-in to future behavior

# Search function
def search_data(student_id, course_id):
    result = data[(data['student_id'] == student_id) & (data['course_id'] == course_id)]
    columns = ['student_id', 'course_id', 'attendance_percentage', 'assignments', 'midterm', 'quiz1', 'quiz2', 'status']
    if not result.empty:
        # Ensure all columns are present, even if missing from merge
        for col in columns:
            if col not in result.columns:
                result[col] = pd.NA
        result = result[columns].infer_objects(copy=False)  # Fix object dtype
        if model is not None:
            try:
                # Force all feature_names columns, impute with numeric values
                X_search = pd.DataFrame(index=result.index, columns=feature_names, dtype='float64')
                for col in feature_names:
                    if col in result.columns:
                        X_search[col] = pd.to_numeric(result[col], errors='coerce')
                    else:
                        X_search[col] = pd.NA
                medians = data[feature_names].median().astype('float64')
                X_search = X_search.fillna({
                    'attendance_percentage': medians['attendance_percentage'],
                    'assignments': medians['assignments'],
                    'midterm': medians['midterm'],
                    'quiz1': medians['quiz1'],
                    'quiz2': medians['quiz2'],
                    'difficulty': 1.0,
                    'department': medians['department']
                }).infer_objects(copy=False)  # Fix object dtype after fillna
                X_search_scaled = scaler.transform(X_search)
                result['Predicted_Probability'] = model.predict_proba(X_search_scaled)[:, 1]
                result['Predicted_Status'] = result['Predicted_Probability'].apply(lambda x: 'Pass' if x >= 0.5 else 'Fail')
            except Exception as e:
                print(f"Prediction failed: {e}")
    return result

# Widgets
student_id_input = widgets.IntText(description='Student ID:')
course_id_input = widgets.IntText(description='Course ID:')
button = widgets.Button(description='Search')
output = widgets.Output(layout={'width': '100%', 'height': 'auto'})  # Adjust layout

def on_button_clicked(b):
    with output:
        output.clear_output()
        student_id = student_id_input.value
        course_id = course_id_input.value
        result = search_data(student_id, course_id)
        if result.empty:
            print("No matching records found.")
        else:
            display(result.style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}]))

button.on_click(on_button_clicked)
display(student_id_input, course_id_input, button, output)

IntText(value=0, description='Student ID:')

IntText(value=0, description='Course ID:')

Button(description='Search', style=ButtonStyle())

Output(layout=Layout(height='auto', width='100%'))